## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    val_dr=dict_val['hist_val'].copy()
    val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins
    
    sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2
    
    size=len(dict_val['hist_val'])
    l1,l2=int(size*0.3),int(size*0.7)
    keys=['chi_1a','chi_1b','chi_1c','chi_1']
    
    for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
        chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})
    
    idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
#     chisqr_dict.update({'chi_sqr1':})
    
    chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
    chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data
    
    idx=60
    spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})
    
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_err'][:idx]**2)})
    
    return chisqr_dict

In [6]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[10000:20000][:,0,:,:]
print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(10000, 128, 128)


In [7]:
df_runs=pd.DataFrame([])

In [8]:
### Load raw images ( a different set of input images for comparison)
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/train.npy'})
images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:5000][:,:,:,0])
dict1['num_img']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


In [9]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['run5_fixed_cosmology','run6_fixed_cosmology','run7_no_truncated_normal','run8_no_truncated_normal']
prefix='keras_'

for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
    key=prefix+strg
    fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:5000]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


In [10]:
# # ### Extract a few images generated by Lbann
# parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/generate_images/'
# # fldr='20200806_065947_gen_img_exagan/'
# prefix='lbann_gen_'

# lst=['20200806_070111_gen_img_exagan/']
# names=['1_b64']

# for strg,fldr in zip(names,lst):
#     f_strg=parent_dir+fldr+'dump_outs/trainer0/model0/sgd.testing.epoch.*.step.*_gen_img_instance1_activation_output0.npy'
#     f_list=glob.glob(f_strg)
#     print(f_list)
#     key=prefix+strg
# #     fname=parent_dir+fldr+'/models/gen_imgs.npy'
#     dict1={'label':key,'fname':f_strg}
#     arr=[np.load(fname)[:,0,:,:] for fname in f_list]
#     images=np.vstack(arr)
#     dict1['num_img']=images.shape[0]
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
#     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)

# # print(s_lbann.shape,np.max(s_lbann))


In [11]:
### Extract some LBANN training run data

parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072639_bsize256_1gpurun/dump_outs/trainer0/model0/'
lst=[(19, 17461), (19, 17457), (18, 15886), (18, 15887)]
prefix='lbann_train_1_'
for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_img']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


{'label': 'lbann_train_1_19-17461', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072639_bsize256_1gpurun/dump_outs/trainer0/model0/sgd.training.epoch.19.step.17461_gen_img_instance1_activation_output0.npy', 'num_img': 256}
{'label': 'lbann_train_1_19-17457', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072639_bsize256_1gpurun/dump_outs/trainer0/model0/sgd.training.epoch.19.step.17457_gen_img_instance1_activation_output0.npy', 'num_img': 256}
{'label': 'lbann_train_1_18-15886', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072639_bsize256_1gpurun/dump_outs/trainer0/model0/sgd.training.epoch.18.step.15886_gen_img_instance1_activation_output0.npy', 'num_img': 256}
{'label': 'lbann_train_1_18-15887', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072639_bsize256_1gpurun/dump_outs/trainer0/model0/sgd.training.epoch.18.step.15887_gen_img_instance1_activatio

In [12]:
parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072615_bsize256_8gpurun/dump_outs/trainer0/model0/'
lst=[(19, 17258), (16, 14798), (18, 15860), (17, 15532)]
prefix='lbann_train_2_'
for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_img']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)

{'label': 'lbann_train_2_19-17258', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072615_bsize256_8gpurun/dump_outs/trainer0/model0/sgd.training.epoch.19.step.17258_gen_img_instance1_activation_output0.npy', 'num_img': 256}
{'label': 'lbann_train_2_16-14798', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072615_bsize256_8gpurun/dump_outs/trainer0/model0/sgd.training.epoch.16.step.14798_gen_img_instance1_activation_output0.npy', 'num_img': 256}
{'label': 'lbann_train_2_18-15860', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072615_bsize256_8gpurun/dump_outs/trainer0/model0/sgd.training.epoch.18.step.15860_gen_img_instance1_activation_output0.npy', 'num_img': 256}
{'label': 'lbann_train_2_17-15532', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_072615_bsize256_8gpurun/dump_outs/trainer0/model0/sgd.training.epoch.17.step.15532_gen_img_instance1_activatio

In [13]:

parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122250_bsize64_8gpurun/dump_outs/trainer0/model0/'
lst=[(6, 24258), (6, 23967), (9, 32897), (5, 20355),(8,28864)]
prefix='lbann_train_3_'
for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_img']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)

{'label': 'lbann_train_3_6-24258', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122250_bsize64_8gpurun/dump_outs/trainer0/model0/sgd.training.epoch.6.step.24258_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_3_6-23967', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122250_bsize64_8gpurun/dump_outs/trainer0/model0/sgd.training.epoch.6.step.23967_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_3_9-32897', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122250_bsize64_8gpurun/dump_outs/trainer0/model0/sgd.training.epoch.9.step.32897_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_3_5-20355', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122250_bsize64_8gpurun/dump_outs/trainer0/model0/sgd.training.epoch.5.step.20355_gen_img_instance1_activation_output0.npy',

In [14]:

parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122131_bsize64_1gpurun/dump_outs/trainer0/model0/'
lst=[(6, 21560), (5, 18439), (6, 22838), (6, 23443)]
prefix='lbann_train_4_'
for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_img']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)

{'label': 'lbann_train_4_6-21560', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122131_bsize64_1gpurun/dump_outs/trainer0/model0/sgd.training.epoch.6.step.21560_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_4_5-18439', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122131_bsize64_1gpurun/dump_outs/trainer0/model0/sgd.training.epoch.5.step.18439_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_4_6-22838', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122131_bsize64_1gpurun/dump_outs/trainer0/model0/sgd.training.epoch.6.step.22838_gen_img_instance1_activation_output0.npy', 'num_img': 64}
{'label': 'lbann_train_4_6-23443', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122131_bsize64_1gpurun/dump_outs/trainer0/model0/sgd.training.epoch.6.step.23443_gen_img_instance1_activation_output0.npy',

In [15]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'fname', 'hist_bin_centers', 'hist_err',
       'hist_val', 'label', 'num_img', 'spec_err', 'spec_val'],
      dtype='object')

In [16]:
col_list=['label','fname', 'hist_bin_centers', 'hist_err', 'hist_val','spec_err', 'spec_val','chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2',
       'chi_imgvar', 'chi_spec1', 'chi_spec2', 'num_img']
df_runs=df_runs.reindex(columns=col_list)

In [17]:
df_runs

,label,fname,hist_bin_centers,hist_err,hist_val,spec_err,spec_val,chi_1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_img
0,raw,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0009205326193552958, 0.0006649037769392328,...","[1.11815584318565, 0.9598541797164788, 0.25895...","[74699.29979158258, 647.5545866649256, 321.540...","[199584026.1747691, 88260.19874135335, 61820.3...",0.000283,0.000005,0.000044,0.000234,0.000001,1.420067,0.000242,51.517680,5000.0
1,keras_0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.000950897267220397, 0.0007432950464243691, ...","[1.1144061578885003, 0.9654176425546366, 0.260...","[72411.96059444547, 592.6523285039614, 302.725...","[198324592.80327728, 84490.05963772327, 60409....",0.000216,0.000072,0.000044,0.000100,0.000057,1.421093,0.038402,17056.945818,5000.0
2,keras_1,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.000966222619487323, 0.000705712698378416, 0...","[1.1089867361063308, 0.9722551246360929, 0.263...","[77451.23967967232, 664.8515508009995, 340.688...","[198734419.12210166, 87154.8526556468, 62094.4...",0.000777,0.000470,0.000218,0.000089,0.000291,1.477033,0.045621,20711.669490,5000.0
3,keras_2,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0009908565295538548, 0.0007354736753678014,...","[1.1458412940016856, 0.9261597979507101, 0.253...","[78867.60127438763, 605.9550411012389, 318.628...","[200495136.2401638, 82535.87758599907, 58592.2...",0.002760,0.002612,0.000038,0.000109,0.001928,1.463532,0.021669,5890.388308,5000.0
4,keras_3,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0009237121372736933, 0.0006931884637765809,...","[1.0966250192317974, 0.9833723445781462, 0.270...","[72430.28920896284, 623.1285179668438, 314.323...","[197651993.2978646, 87546.65632453177, 62182.7...",0.002429,0.001642,0.000558,0.000229,0.001208,1.421895,0.033354,18905.534564,5000.0
5,lbann_train_1_19-17461,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0036750265579990047, 0.002902691294628473, ...","[1.0965382112534172, 0.9844441354538006, 0.266...","[272772.9683255477, 2734.9202509442475, 1256.0...","[196725754.39234307, 84178.67804340739, 59797....",0.022216,0.001989,0.001038,0.019190,0.001218,5.979433,0.366805,12110.398385,256.0
6,lbann_train_1_19-17457,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.004023873921628211, 0.003249668512702734, 0...","[1.1167992829280495, 0.9691623836783614, 0.257...","[292602.89844753745, 2609.3209692794076, 1377....","[198161530.7046417, 81979.26900136657, 58461.7...",0.007114,0.000741,0.001956,0.004416,0.000165,6.116924,0.600518,17350.918078,256.0
7,lbann_train_1_18-15886,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.004140050622083048, 0.0031932407578727892, ...","[1.0360511791607536, 1.0386597604657897, 0.299...","[306647.1646329731, 2691.297988723453, 1574.21...","[193844608.29144102, 88245.01863658987, 62168....",0.042161,0.027990,0.006080,0.008090,0.015542,6.125851,0.112922,2708.756358,256.0
8,lbann_train_1_18-15887,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.00434345262356535, 0.003322326250328145, 0....","[1.0480717000576765, 1.0240315289178346, 0.294...","[325187.58320760675, 3412.5402828479823, 1556....","[194343398.9717024, 92760.26333999448, 64979.9...",0.050722,0.022347,0.004584,0.023792,0.011124,6.584180,0.115207,2646.104592,256.0
9,lbann_train_2_19-17258,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.006112924240651057, 0.00421078517122679, 0....","[1.105911

### Compare chi-squares

In [18]:
chi_keys=['label','chi_1','chi_spec1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'num_img','fname']
df_runs[chi_keys]

,label,chi_1,chi_spec1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,num_img,fname
0,raw,0.000283,0.000242,0.000005,0.000044,0.000234,0.000001,1.420067,51.517680,5000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
1,keras_0,0.000216,0.038402,0.000072,0.000044,0.000100,0.000057,1.421093,17056.945818,5000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
2,keras_1,0.000777,0.045621,0.000470,0.000218,0.000089,0.000291,1.477033,20711.669490,5000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
3,keras_2,0.002760,0.021669,0.002612,0.000038,0.000109,0.001928,1.463532,5890.388308,5000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
4,keras_3,0.002429,0.033354,0.001642,0.000558,0.000229,0.001208,1.421895,18905.534564,5000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,lbann_train_1_19-17461,0.022216,0.366805,0.001989,0.001038,0.019190,0.001218,5.979433,12110.398385,256.0,/global/cscratch1/sd/vpa/proj/cosmogan/results...
6,lbann_train_1_19-17457,0.007114,0.600518,0.000741,0.001956,0.004416,0.000165,6.116924,17350.918078,256.0,/global/cscratch1/sd/vpa/proj/cosmogan/results...
7,lbann_train_1_18-15886,0.042161,0.112922,0.027990,0.006080,0.008090,0.015542,6.125851,2708.756358,256.0,/global/cscratch1/sd/vpa/proj/cosmogan/results...
8,lbann_train_1_18-15887,0.050722,0.115207,0.022347,0.004584,0.023792,0.011124,6.584180,2646.104592,256.0,/global/cscratch1/sd/vpa/proj/cosmogan/results...
9,lbann_train_2_19-17258,0.005790,0.287492,0.001439,0.001546,0.002806,0.000403,7.672626,3516.360327,256.0,/global/cscratch1/sd/vpa/proj/cosmogan/results...


In [19]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec'],"Invalid mode %s"%(plot_type)

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    
#     plt.xscale('symlog',linthreshx=30)

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')

In [20]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec']))

interactive(children=(SelectMultiple(description='labels_list', options=('raw', 'keras_0', 'keras_1', 'keras_2…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>